In [1]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.keras import layers, models
from sklearn.metrics import precision_score, recall_score, accuracy_score
import cv2
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:
!unzip BT_50.zip -d /content/

Archive:  BT_50.zip
   creating: /content/BT_50/
   creating: /content/BT_50/Testing/
   creating: /content/BT_50/Testing/glioma/
  inflating: /content/BT_50/Testing/glioma/Te-glTr_0000.jpg  
  inflating: /content/BT_50/Testing/glioma/Te-glTr_0001.jpg  
  inflating: /content/BT_50/Testing/glioma/Te-glTr_0002.jpg  
  inflating: /content/BT_50/Testing/glioma/Te-glTr_0003.jpg  
  inflating: /content/BT_50/Testing/glioma/Te-glTr_0004.jpg  
  inflating: /content/BT_50/Testing/glioma/Te-glTr_0005.jpg  
  inflating: /content/BT_50/Testing/glioma/Te-glTr_0006.jpg  
  inflating: /content/BT_50/Testing/glioma/Te-glTr_0007.jpg  
  inflating: /content/BT_50/Testing/glioma/Te-glTr_0008.jpg  
  inflating: /content/BT_50/Testing/glioma/Te-glTr_0009.jpg  
  inflating: /content/BT_50/Testing/glioma/Te-gl_0160.jpg  
  inflating: /content/BT_50/Testing/glioma/Te-gl_0161.jpg  
  inflating: /content/BT_50/Testing/glioma/Te-gl_0162.jpg  
  inflating: /content/BT_50/Testing/glioma/Te-gl_0163.jpg  
  inflatin

In [3]:
image_size = (224, 224)
batch_size = 32

In [23]:
# train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
#     rescale=1./255,
#     rotation_range=40,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     fill_mode='nearest',
#     brightness_range=[0.8, 1.2]  # Random brightness adjustment
# )


In [31]:
# train_data = train_datagen.flow_from_directory(
#     "BT_50/Testing",
#     target_size=(224, 224),  # Resize all images to 224x224
#     batch_size=32,  # Number of images to return in each batch
#     class_mode='sparse',  # Use sparse integer labels for multi-class classification
#     shuffle=True  # Shuffle the data
# )

Found 600 images belonging to 4 classes.


In [35]:
train_data = tf.keras.preprocessing.image_dataset_from_directory(
    "BT_50/Training",
    image_size=image_size,
    batch_size=batch_size,
    label_mode="int",
    shuffle=True
)

Found 2640 files belonging to 4 classes.


In [36]:
# Load Testing Data (Completely Unseen)
test_data = tf.keras.preprocessing.image_dataset_from_directory(
    "BT_50/Testing",
    image_size=image_size,
    batch_size=batch_size,
    label_mode="int",
    shuffle=False
)

Found 600 files belonging to 4 classes.


In [7]:
# model = models.Sequential([
#     layers.Input(shape=(224, 224, 3)),  # Explicit input layer
#     layers.Conv2D(32, (3, 3), activation='relu'),
#     layers.MaxPooling2D((2, 2)),
#     layers.Conv2D(64, (3, 3), activation='relu'),
#     layers.MaxPooling2D((2, 2)),
#     layers.Conv2D(128, (3, 3), activation='relu'),
#     layers.MaxPooling2D((2, 2)),
#     layers.Flatten(),
#     layers.Dense(128, activation='relu'),
#     layers.Dense(4, activation='softmax')
# ])
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [37]:
# # Define the custom CNN model
# model = models.Sequential([
#     # 1st convolutional block
#     layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3), padding='same'),
#     layers.MaxPooling2D((2, 2)),

#     # 2nd convolutional block
#     layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
#     layers.MaxPooling2D((2, 2)),

#     # 3rd convolutional block
#     layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
#     layers.MaxPooling2D((2, 2)),

#     # Flatten the output of convolutional layers
#     layers.Flatten(),

#     # Fully connected (Dense) layer
#     layers.Dense(512, activation='relu'),

#     # Dropout for regularization
#     layers.Dropout(0.5),

#     # Output layer for multi-class classification (4 classes)
#     layers.Dense(4, activation='softmax')
# ])

# # Compile the model
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# # Display the model summary
# model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_15 (Conv2D)                   │ (None, 224, 224, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_15 (MaxPooling2D)      │ (None, 112, 112, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_16 (Conv2D)                   │ (None, 112, 112, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_16 (MaxPooling2D)      │ (None, 56, 56, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_17 (Conv2D)                   │ (None, 56, 56, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_17 (MaxPooling2D)      │ (None, 28, 28, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_5 (Flatten)                  │ (None, 100352)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 512)                 │      51,380,736 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 4)                   │           2,052 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 51,476,036 (196.37 MB)

 Trainable params: 51,476,036 (196.37 MB)

 Non-trainable params: 0 (0.00 B)

In [42]:
model = models.Sequential([
    layers.Conv2D(16, (3, 3), activation='relu', input_shape=(224, 224, 3), padding='same'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),

    layers.Dense(4, activation='softmax')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [43]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_18 (Conv2D)                   │ (None, 224, 224, 16)        │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_18 (MaxPooling2D)      │ (None, 112, 112, 16)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_19 (Conv2D)                   │ (None, 112, 112, 32)        │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_19 (MaxPooling2D)      │ (None, 56, 56, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_20 (Conv2D)                   │ (None, 56, 56, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_20 (MaxPooling2D)      │ (None, 28, 28, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_6 (Flatten)                  │ (None, 50176)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 256)                 │      12,845,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 4)                   │           1,028 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 12,869,924 (49.09 MB)

 Trainable params: 12,869,924 (49.09 MB)

 Non-trainable params: 0 (0.00 B)

In [44]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

# Train the model
# model.fit(train_data, epochs=20, validation_data=test_data,
#           callbacks=[early_stopping, reduce_lr])
model.fit(train_data, epochs=20, validation_data=test_data)

Epoch 1/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 14s 108ms/step - accuracy: 0.5339 - loss: 72.9268 - val_accuracy: 0.7483 - val_loss: 0.9640
Epoch 2/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 13s 55ms/step - accuracy: 0.8438 - loss: 0.4198 - val_accuracy: 0.7950 - val_loss: 1.0462
Epoch 3/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 5s 56ms/step - accuracy: 0.9130 - loss: 0.2581 - val_accuracy: 0.7983 - val_loss: 1.3831
Epoch 4/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 5s 57ms/step - accuracy: 0.9403 - loss: 0.1679 - val_accuracy: 0.8117 - val_loss: 1.5827
Epoch 5/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 5s 55ms/step - accuracy: 0.9600 - loss: 0.1098 - val_accuracy: 0.8233 - val_loss: 1.8491
Epoch 6/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 5s 62ms/step - accuracy: 0.9778 - loss: 0.0693 - val_accuracy: 0.8267 - val_loss: 1.5407
Epoch 7/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 9s 49ms/step - accuracy: 0.9811 - loss: 0.0623 - val_accuracy: 0.8417 - val_loss: 2.2709
Epoch 8/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 6s 57ms/step - accuracy: 0.9821 - loss: 0.0493 - val_accuracy: 0.8417

In [45]:
from sklearn.metrics import classification_report

y_pred = model.predict(test_data)
y_pred_classes = np.argmax(y_pred, axis=1)
true_labels = np.concatenate([y.numpy() for x, y in test_data], axis=0)
print(classification_report(true_labels, y_pred_classes))

19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step
              precision    recall  f1-score   support

           0       0.79      0.89      0.83       150
           1       0.74      0.89      0.81       150
           2       0.95      0.97      0.96       150
           3       0.97      0.63      0.76       150

    accuracy                           0.84       600
   macro avg       0.86      0.84      0.84       600
weighted avg       0.86      0.84      0.84       600

